In [9]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import Image, ImageTk
import cv2
import numpy as np

# Assuming you have these files and they are set up correctly
from TPR_SVM_PCA_Y import best_svm_classifier, pca as svm_pca
from TPR_KNN_PCA_Y import best_knn_classifier, pca as knn_pca

# Update your classes based on the actual categories
classes = {
    0: 'Part A',
    1: 'Part B',
    2: 'Part C',
    3: 'Part D'
}

def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        classify(file_path)

def classify(file_path):
    if classification_method.get() == "SVM":
        classify_svm(file_path)
    elif classification_method.get() == "KNN":
        classify_knn(file_path)
    else:
        print("Invalid classification method selected.")

def classify_svm(file_path):
    try:
        display_image_on_gui(file_path)

        image = Image.open(file_path).convert("L")
        image = image.resize((150, 150))
        image_array = np.array(image)
        image_array = image_array.reshape(1, -1)
        image_array = image_array / 255.0

        image_pca = svm_pca.transform(image_array)  # Make sure to use the correct PCA
        pred = best_svm_classifier.predict(image_pca)[0]
        result_label.configure(text=f'Predicted: {classes[pred]}', fg='white')

        parts_count = count_parts(file_path)
        count_label.configure(text=f'Number of parts: {parts_count}', fg='white')
    except Exception as e:
        result_label.configure(text="Error: " + str(e), fg='red')
        count_label.configure(text="")

def classify_knn(file_path):
    try:
        display_image_on_gui(file_path)

        image = Image.open(file_path).convert("L")
        image = image.resize((150, 150))
        image_array = np.array(image)
        image_array = image_array.reshape(1, -1)
        image_array = image_array / 255.0

        image_pca = knn_pca.transform(image_array)  # Make sure to use the correct PCA
        pred = best_knn_classifier.predict(image_pca)[0]
        result_label.configure(text=f'Predicted: {classes[pred]}', fg='white')

        parts_count = count_parts(file_path)
        count_label.configure(text=f'Number of parts: {parts_count}', fg='white')
    except Exception as e:
        result_label.configure(text="Error: " + str(e), fg='red')
        count_label.configure(text="")

def display_image_on_gui(file_path):
    original_image = Image.open(file_path)
    original_image = original_image.resize((300, 260))
    img_original = ImageTk.PhotoImage(original_image)
    image_label.configure(image=img_original)
    image_label.image = img_original

def count_parts(image_path, threshold=128, min_area=500):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply a binary threshold to the image
    _, binary_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY_INV)
    
    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter contours by minimum area
    parts_count = 0
    for contour in contours:
        if cv2.contourArea(contour) > min_area:
            parts_count += 1
    
    # Ensure parts count is within the range of 1 to 3
    return max(1, min(parts_count, 3))

# Create GUI window
window = tk.Tk()
window.title("Recognition System")
window.geometry("800x600")
window.configure(background='#23272A')

# Create label for title
title_label = tk.Label(window, text="Parts Recognition and Counting System", font=("Helvetica", 20, "bold"), bg='#23272A', fg='#ECECEC')
title_label.pack(pady=20)

# Create frame for image display
image_frame = tk.Frame(window, bg='#23272A')
image_frame.pack(pady=10)

# Create label to display image
image_label = tk.Label(image_frame, bg='#23272A', fg='white')
image_label.pack()

# Create frame for classification result
result_frame = tk.Frame(window, bg='#23272A')
result_frame.pack(pady=20)

# Create label to display classification result
result_label = tk.Label(result_frame, text="", font=("Helvetica", 18), bg='#23272A', fg='white')
result_label.pack()

# Create frame for part counting result
count_frame = tk.Frame(window, bg='#23272A')
count_frame.pack(pady=10)

# Create label to display part counting result
count_label = tk.Label(count_frame, text="", font=("Helvetica", 18), bg='#23272A', fg='white')
count_label.pack()

# Create a radio button group for classification method selection
method_frame = tk.Frame(window, bg='#23272A')
method_frame.pack(pady=10)

classification_method = tk.StringVar()
svm_radio_button = tk.Radiobutton(method_frame, text="SVM", variable=classification_method, value="SVM", font=("Helvetica", 12), bg='#23272A', fg='#ECECEC', selectcolor='#23272A', activebackground='#23272A', activeforeground='#ECECEC')
svm_radio_button.pack(side=tk.LEFT, padx=10)
knn_radio_button = tk.Radiobutton(method_frame, text="KNN", variable=classification_method, value="KNN", font=("Helvetica", 12), bg='#23272A', fg='#ECECEC', selectcolor='#23272A', activebackground='#23272A', activeforeground='#ECECEC')
knn_radio_button.pack(side=tk.LEFT, padx=10)

# Create button for image upload
upload_button = tk.Button(window, text="Upload Image", command=upload_image, font=("Helvetica", 14), bg='#ECECEC', fg='#23272A', relief=tk.RAISED)
upload_button.pack(pady=20)

# Start the GUI loop
window.mainloop()
